## Setup

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'st-data-project'

In [2]:
REGION = 'us-central1'

**Packages**

In [4]:
from google.cloud import storage
from google.cloud import bigquery

import pandas as pd

import datetime

In [14]:
BUCKET = PROJECT_ID + '-mlops-in-gcp-' + datetime.datetime.today().strftime("%Y%m%d%H%M%S")

In [15]:
BUCKET

'st-data-project-mlops-in-gcp-20230922121338'

---
<a id = 'permissions'></a>
## Service Account & Permissions

This notebook instance is running as a service account in GCP.  This service account will also be used to run other services in Vertex AI like training jobs and pipelines.  The service account will need permission to interact with object in Cloud Storage which requires the role ([roles/storage.objectAdmin](https://cloud.google.com/storage/docs/access-control/iam-roles)).  

Get the current service account:

In [16]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'991309255301-compute@developer.gserviceaccount.com'

Enable the Cloud Resource Manager API:

In [17]:
!gcloud services enable cloudresourcemanager.googleapis.com

List the service accounts current roles:

In [18]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/aiplatform.notebookRuntimeAdmin
roles/bigquery.admin
roles/editor
roles/storage.objectAdmin


If the resulting list is missing `roles/storage.objectAdmin` or another role that contains this permission, like the basic role `roles/owner`, then it will need to be added for the service account. Use these instructions to complete this:

In [18]:
print(f'Go To IAM in the Google Cloud Console:\nhttps://console.cloud.google.com/iam-admin/iam?orgonly=true&project={PROJECT_ID}&supportedpurview=organizationId')

Go To IAM in the Google Cloud Console:
https://console.cloud.google.com/iam-admin/iam?orgonly=true&project=statmike-mlops-349915&supportedpurview=organizationId


From the console link above, or by going to https:/console.cloud.google.com and navigating to "IAM & Admin > IAM":
- Locate the row for the service account listed above: `<project number>-compute@developer.gserviceaccount.com`
- Under the `inheritance` column click the pencil icon to edit roles
- In the fly over menu, under `Assign roles` select `Add Another Role`
- Click the `Select a role` box and type `Storage Object Admin`, then select `Storage Object Admin`
- Click Save
- Rerun the list of services below and verify the role has been added:

In [19]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
roles/bigquery.admin
roles/owner
roles/run.admin
roles/storage.objectAdmin


---
## Install KFP
If you get an error after a step, rerun it.  The dependecies sometimes resolve.
- [Install the Kubeflow Pipelines SDK](https://www.kubeflow.org/docs/components/pipelines/v1/sdk/install-sdk/)

In [1]:
!pip install kfp -U -q

In [2]:
!pip install google-cloud-pipeline-components -U -q

---
## Update AIPlatform Package:

The `google-cloud-aiplatform` package updates frequently.  Update it for latest functionality.

- [aiplatform Python Client](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform)
- [GitHub Repo for api-common-protos](https://github.com/googleapis/api-common-protos)

For a better understanding of the Vertex AI APIs client, version, and layers please review the tip here [aiplatform_notes.md](../Tips/aiplatform_notes.md).

In [5]:
!pip install googleapis-common-protos -U -q

In [6]:
!pip install google-cloud-aiplatform -U -q

In [7]:
from google.cloud import aiplatform
aiplatform.__version__

'1.21.0'

In [3]:
!pip install --upgrade google-cloud-pipeline-components

  Obtaining dependency information for google-cloud-pipeline-components from https://files.pythonhosted.org/packages/91/78/dadce90c2a413e3b5eba89c78df9b47b7227d5c49652577fd43fab1d75ae/google_cloud_pipeline_components-2.4.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: kfp-pipeline-spec
    Found existing installation: kfp-pipeline-spec 0.1.16
    Uninstalling kfp-pipeline-spec-0.1.16:
      Successfully uninstalled kfp-pipeline-spec-0.1.16
  Attempting uninstall: google-cloud-pipeline-components
    Found existing installation: google-cloud-pipeline-components 2.3.1
    Uninstalling google-cloud-pipeline-components-2.3.1:
      Successfully uninstalled google-cloud-pipeline-components-2.3.1
